In [25]:

mainUrl = "http://www.sec.gov/Archives/edgar/data/"
CIK = input("Enter CIK (eg :0000051143)")
 # DAN has to come from some search results as per problem case
DAN = input("Enter document accession number -DAN (eg :000005114313000007 )")
CIK = CIK.strip().strip("0")
DAN = DAN.strip()
partCIK = DAN[0:10]+"-"
partDAN = DAN[10:12]+"-"
lastPart = DAN[12:18]+"-index.html"

completeURL = mainUrl+CIK+"/"+DAN+"/"+partCIK+partDAN+lastPart
print(completeURL)


Enter CIK (eg :0000051143)0000051143
Enter document accession number -DAN (eg :000005114313000007 )000005114313000007 
http://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.html
